# Import

In [1]:
import json
import pandas as pd
import numpy as np
# import spacy
import matplotlib.pyplot as plt
import ast

pd.set_option('display.max_colwidth', None)

In [2]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict

In [3]:
# %cd ../../
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../..'))
from utils.prompting import *
from utils.preprocessing import preprocessing_comment_data
import time
from multiprocessing import Pool
from os import listdir
import ast

In [4]:
import openai
import os
import pandas as pd
import time

openai.api_key = "EMPTY" # Not support yet
openai.api_base = "http://localhost:8000/v1"

model = "vicuna-7b-v1.3"
prompt = "Once upon a time"

In [5]:
# create a completion
completion = openai.Completion.create(model=model, prompt=prompt, max_tokens=64)
# print the completion
print(prompt + completion.choices[0].text)

Once upon a time, there was a little girl who loved to read. She read every day, sometimes for hours on end. She read books about fairies, and dragons, and mermaids. She read books about animals, and adventure, and mystery. And she loved to read about princesses, particularly the


# Load SPACE dataset

In [6]:
space_df = pd.read_json("../../data/space/input_reviews/space_summ.json")
data = space_df.to_json(orient='records')
data = ast.literal_eval(data)
review_df = pd.json_normalize(
    data, "reviews", ['entity_id', 'entity_name']
)
review_df

,review_id,sentences,rating,entity_id,entity_name
0,UR59977476,"[We stayed here on a lay over home from Cancun., It was great to have a comfortable bed and room on our final night of holidays., The kids loved the pool which was warmer than the ones at the resort in Cancun which we could not believe as we were in Seattle!, The staff was friendly and we appreciated the cookies after a long flight when we were waiting to check inn., Just a nice touch!, Shuttle was convenient and would definitely stay here again.]",5,100597,Doubletree by Hilton Seattle Airport
1,UR116963461,"[I reserved this room through an affiliate travel booking platform and got a great room rate., Aprox., 75.00 (not including taxes) for a full day stay., Late check in was effortless and was greeted with a warmed cookie., I think it is fair to mention it costs to use internet in your room but for business travlers I suppose that is an expected cost., Great friendly staff took the edge off our long day of intercontinental travel., I would choose this hotel again as our stop over.]",4,100597,Doubletree by Hilton Seattle Airport
2,UR115823712,"[We flew into SEA TAC for a few days before our cruise and our travel agent recommended this hotel., The hotel was clean, beds were fine, hotel is located across the street from the air port, was within walking distance to a Denny's, Jack in the Box, and a steak place., Room was ready very early in the morning for us., The kids liked the pool and I was able to do laundry before the cruise., It had easy access to the light rail located at the airport (we took the DT shuttle over), and was next to an Enterprise car rental., Lots of conveniences close by., Downtown Seattle was about a 30 minute ride by light rail service (cost of $16 for the 4 of us) or $40 cab ride to the pier., Hope this helps.]",4,100597,Doubletree by Hilton Seattle Airport
3,UR115912623,"[This is a large hotel and I expected it to be large and impersonal., On the contrary., It has ""small hotel"" service, meaning that you don't feel lost and they treat you well., Several fireplaces and seating areas in the lobby make it comfortable., I had a reservation for a ""mountain view"" room., Initially, I had a view of the airport., I asked for a different room and was given a much better view on the other side of the main section of the hotel., Very nice pool area although in cool, rainy Seattle I didn't get a chance to swim., The food was good although a bit pricey., The service in both the casual restaurant at breakfast and the dining room in the evening was efficient., Spacious room and very comfortable bedding., The room to which I was initially assigned had a broken lamp and a rip in the carpet., The second room was nicer., Looks like they are trying to keep up with on-going updating and maintenance., I stayed here four nights for a conference and was actually pleasantly surprised by the overall hotel experience., Lots of airline personnel stay here as well., And yes, we like the chocolate chip cookies at check-in.]",4,100597,Doubletree by Hilton Seattle Airport
4,UR2825386,"[I stayed here on November 7-8, 2004., When I read the previous reviews, I asked if my reservation was for a wing or a tower room., It was for a wing room but the reservation staff courteously changed it to a tower room., Access to the tower is by external glass elevators., My room was large, quiet and had a small balcony., There was a stain on the carpet but otherwise it was clean and well maintained., The room was equipped with a coffee maker, iron & ironing board and blow dryer and there were 2 sinks - one inside the bathroom and one outside., The room did not have a refrigerator or microwave; the high speed internet connection in the room cost about $10., There is no complimentary breakfast; there is a coffee shop and an espresso bar that also sells yogurt, bagels and pastries., The hotel has a complimentary airport shuttle that runs every 15 minutes, 24 hours a day., I was attending a

## Sample SPACE dataset

In [7]:
sampled_review_df = review_df[review_df['sentences'].str.len() <= 15]
review_counts = sampled_review_df['entity_name'].value_counts()
sampled_review_df = sampled_review_df[sampled_review_df['entity_name'].isin(review_counts.index[:10])]

In [8]:
sampled_review_df.shape

(946, 5)

In [9]:
sampled_comment_df = sampled_review_df.explode(['sentences'])
sampled_comment_df = sampled_comment_df[sampled_comment_df['sentences'].str.len() >= 10]
sampled_comment_df

,review_id,sentences,rating,entity_id,entity_name
1201,UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma
1201,UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma
1201,UR70384224,But clean.,5,1510471,UNA Hotel Roma
1201,UR70384224,- Bed was comfortable - Bathroom was modern and clean - Shower doesn't totally close.,5,1510471,UNA Hotel Roma
1201,UR70384224,- Temperature is controlled from the front desk.,5,1510471,UNA Hotel Roma
...,...,...,...,...,...
4899,UR82736454,"I'm a chipped paint snob, so it is a bit of a turn off when you pay several hundred dollars and walk into a room that needs some repairs.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel"
4899,UR82736454,i don't know why those details are so commonly overlooked by upscale hotels.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel"
4899,UR82736454,The best part of the hotel is the 7th floor rooftop deck.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel"
4899,UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel"


## Preprocess

In [10]:
sampled_comment_df = preprocessing_comment_data(sampled_comment_df)

In [11]:
sampled_comment_df.to_pickle("../../data/space/input_reviews/space_sampled.pkl")

# Inference

## Setup

In [12]:
import time

start_time = time.time()
comment = "The food is excellent but the portion is so small ."
print(comment)
response = get_absa_completion(comment)
print(response)
print("TIME ELAPSED", time.time() - start_time)

The food is excellent but the portion is so small .
Answer: [{'aspect': 'food','sentiment': 'positive'}, {'aspect': 'portion','sentiment': 'negative'}]

Review sentence: The food has great taste but the price is too high and the service
TIME ELAPSED 5.066383361816406


## Prompt-based ABSA

In [13]:
num_workers = 1
root_path = '../../data/space/processed_absa_reviews'

In [14]:
sampled_comment_df['category'] = 'Hotel'

Multi-thread run

In [15]:
inputs = [(root_path, 
           domain, 
           sampled_comment_df[sampled_comment_df['category'] == domain].reset_index(drop=True)
          )
          for domain in sampled_comment_df['category'].unique()]

start_time = time.time()
with Pool(num_workers) as processor:
    data = processor.starmap(prompt_based_absa, inputs)
print("TIME ELAPSED", time.time() - start_time)

Hotel :  Loaded preprocessed file. Done
TIME ELAPSED 0.1592259407043457


In [16]:
processed_df = pd.concat(data)
processed_df.shape

(7453, 7)

## Post-processing

### Fix ' character in the response

In [17]:
import pandas as pd
import re
import ast

In [18]:
mask = processed_df['absa_extractions'].str.contains(": *\'([^':,]*\'+[^':,]*)+\' *,")
processed_df.loc[mask, 'absa_extractions'] = processed_df.loc[mask, 'absa_extractions'].apply(
    lambda x: re.sub(r"(: *)\'((?:[^':,]*\'+[^':,]*)+)\'( *,)", r'\1"\2"\3', x))

/tmp/ipykernel_6288/3278709864.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = processed_df['absa_extractions'].str.contains(": *\'([^':,]*\'+[^':,]*)+\' *,")


In [19]:
processed_df

,absa_extractions,review_id,sentences,rating,entity_id,entity_name,category
0,"[{'aspect': 'bathroom','sentiment': 'positive'}, {'aspect': 'room','sentiment': 'positive'}]",UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma,Hotel
1,"[{'aspect':'size','sentiment': 'neutral'}]",UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma,Hotel
2,"[{'aspect': 'cleanliness','sentiment': 'neutral'}]",UR70384224,But clean.,5,1510471,UNA Hotel Roma,Hotel
3,"[{'aspect': 'bed','sentiment': 'positive'}, {'aspect': 'bathroom','sentiment': 'positive'}, {'aspect':'shower','sentiment': 'neutral'}]",UR70384224,- Bed was comfortable - Bathroom was modern and clean - Shower doesn't totally close.,5,1510471,UNA Hotel Roma,Hotel
4,"[{'aspect': 'temperature','sentiment': 'neutral'}]",UR70384224,- Temperature is controlled from the front desk.,5,1510471,UNA Hotel Roma,Hotel
...,...,...,...,...,...,...,...
7448,"[{'aspect': 'room','sentiment': 'negative'}]",UR82736454,"I'm a chipped paint snob, so it is a bit of a turn off when you pay several hundred dollars and walk into a room that needs some repairs.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7449,"[{'aspect': 'hotel details','sentiment': 'neutral'}]",UR82736454,i don't know why those details are so commonly overlooked by upscale hotels.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7450,"[{'aspect': 'hotel','sentiment': 'positive'}]",UR82736454,The best part of the hotel is the 7th floor rooftop deck.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7451,"[{'aspect': 'couch','sentiment': 'neutral'}, {'aspect':'sunset','sentiment': 'positive'}, {'aspect': 'boardwalk','sentiment': 'neutral'}, {'aspect': 'cocktail','sentiment': 'positive'}]",UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel


### Correct JSON

In [20]:
mask = processed_df['absa_extractions'].str.startswith("[{'aspect': 'breads', 'cakes', 'pies', 'desserts','sentiment': 'positive'}]")
processed_df[mask]

,absa_extractions,review_id,sentences,rating,entity_id,entity_name,category


In [21]:
for fix_id in processed_df[mask].index.tolist():
    fix_row = processed_df.loc[fix_id]
    processed_df.loc[fix_id, 'absa_extractions'] = fix_json(fix_row['absa_extractions'])

### Normalize JSON into columns

In [22]:
import pandas as pd
import json

In [23]:
processed_df['absa_extractions'] = processed_df['absa_extractions'].str.replace("\n", "")
processed_df['absa_extractions'] = processed_df['absa_extractions'].apply(lambda x: ast.literal_eval(x))
processed_df= processed_df.explode(['absa_extractions'])
processed_df

,absa_extractions,review_id,sentences,rating,entity_id,entity_name,category
0,"{'aspect': 'bathroom', 'sentiment': 'positive'}",UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma,Hotel
0,"{'aspect': 'room', 'sentiment': 'positive'}",UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma,Hotel
1,"{'aspect': 'size', 'sentiment': 'neutral'}",UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma,Hotel
2,"{'aspect': 'cleanliness', 'sentiment': 'neutral'}",UR70384224,But clean.,5,1510471,UNA Hotel Roma,Hotel
3,"{'aspect': 'bed', 'sentiment': 'positive'}",UR70384224,- Bed was comfortable - Bathroom was modern and clean - Shower doesn't totally close.,5,1510471,UNA Hotel Roma,Hotel
...,...,...,...,...,...,...,...
7451,"{'aspect': 'boardwalk', 'sentiment': 'neutral'}",UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7451,"{'aspect': 'cocktail', 'sentiment': 'positive'}",UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7452,"{'aspect': 'food', 'sentiment': 'negative'}",UR82736454,"The food was not very good, so sip your drinks slowly and save your appetite for the restaurants on Abbott Kinney.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7452,"{'aspect': 'drinks', 'sentiment': 'neutral'}",UR82736454,"The food was not very good, so sip your drinks slowly and save your appetite for the restaurants on Abbott Kinney.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel


In [24]:
json_struct = json.loads(processed_df.to_json(orient="records"))    
processed_df = pd.json_normalize(json_struct)
processed_df.columns = [col.replace('absa_extractions.', 'prompt_') for col in processed_df.columns]
processed_df = processed_df.drop(columns=['absa_extractions'])
processed_df = processed_df[pd.notnull(processed_df['prompt_aspect'])]
processed_df

,review_id,sentences,rating,entity_id,entity_name,category,prompt_aspect,prompt_sentiment
0,UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma,Hotel,bathroom,positive
1,UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma,Hotel,room,positive
2,UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma,Hotel,size,neutral
3,UR70384224,But clean.,5,1510471,UNA Hotel Roma,Hotel,cleanliness,neutral
4,UR70384224,- Bed was comfortable - Bathroom was modern and clean - Shower doesn't totally close.,5,1510471,UNA Hotel Roma,Hotel,bed,positive
...,...,...,...,...,...,...,...,...
11310,UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,boardwalk,neutral
11311,UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,cocktail,positive
11312,UR82736454,"The food was not very good, so sip your drinks slowly and save your appetite for the restaurants on Abbott Kinney.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,food,negative
11313,UR82736454,"The food was not very good, so sip your drinks slowly and save your appetite for the restaurants on Abbott Kinney.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,drinks,neutral


### Standardization

In [25]:
import spacy
nlp = spacy.load('en_core_web_sm')
# Standardization
processed_df['prompt_aspect_lemm'] = processed_df['prompt_aspect'].apply(
    lambda aspect: " ".join([token.lemma_ for token in nlp(f'{aspect.lower()}')])
)

/home/an/anaconda3/envs/deep_learning_env_2/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [26]:
processed_df

,review_id,sentences,rating,entity_id,entity_name,category,prompt_aspect,prompt_sentiment,prompt_aspect_lemm
0,UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma,Hotel,bathroom,positive,bathroom
1,UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma,Hotel,room,positive,room
2,UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma,Hotel,size,neutral,size
3,UR70384224,But clean.,5,1510471,UNA Hotel Roma,Hotel,cleanliness,neutral,cleanliness
4,UR70384224,- Bed was comfortable - Bathroom was modern and clean - Shower doesn't totally close.,5,1510471,UNA Hotel Roma,Hotel,bed,positive,bed
...,...,...,...,...,...,...,...,...,...
11310,UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,boardwalk,neutral,boardwalk
11311,UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,cocktail,positive,cocktail
11312,UR82736454,"The food was not very good, so sip your drinks slowly and save your appetite for the restaurants on Abbott Kinney.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,food,negative,food
11313,UR82736454,"The food was not very good, so sip your drinks slowly and save your appetite for the restaurants on Abbott Kinney.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,drinks,neutral,drink


In [27]:
processed_df.to_pickle(f"{root_path}/absa_processed_df.pkl")